In [ ]:
import pandas as pd

# Replace with your CSV file path
csv_file = 'stm32_function_with_tests.csv'

# Read the CSV file
df = pd.read_csv(csv_file)

# Count missing values per column
missing_counts = df.isnull().sum()

# Print the result
print("Number of missing values per column:")
print(missing_counts)


Number of missing values per column:
file_name          0
function_code      0
description        0
test_function    270
dtype: int64


In [ ]:
df_cleaned = df.dropna()

In [ ]:
missing_counts = df_cleaned.isnull().sum()

# Print the result
print("Number of missing values per column:")
print(missing_counts)

Number of missing values per column:
file_name        0
function_code    0
description      0
test_function    0
dtype: int64


In [ ]:
df_cleaned = df_cleaned.reset_index(drop=True)


In [ ]:
import pandas as pd

# Suppose df_cleaned is a pandas DataFrame with the specified columns.
df_code_test = df_cleaned[['function_code', 'test_function']].copy()
df_desc_test = df_cleaned[['description', 'test_function']].copy()


In [ ]:
from huggingface_hub import login
from google.colab import userdata
login(token=userdata.get("HF_TOKEN"))

In [ ]:
import pandas as pd

# Supposons que df_code_test et df_desc_test existent déjà dans ta session

# 1. Renommer les colonnes
df_code_test_ready = df_code_test.rename(columns={
    'function_code': 'input',
    'test_function': 'target'
})

df_desc_test_ready = df_desc_test.rename(columns={
    'description': 'input',
    'test_function': 'target'
})


In [ ]:
from sklearn.model_selection import train_test_split

# Supposons que df_code_test_renamed a les colonnes "input" et "target"
train_df, eval_df = train_test_split(df_code_test_ready, test_size=0.1, random_state=42)

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-small")

# Créer les Dataset directement à partir des DataFrames
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

# Créer un DatasetDict pour regrouper train et eval
dataset = DatasetDict({
    'train': train_dataset,
    'validation': eval_dataset
})

# Tokenisation pour l'input
def tokenize_inputs(batch):
    return tokenizer(
        batch['input'],
        padding="max_length",
        truncation=True,
        max_length=512
    )

# Tokenisation pour la target (labels)
def tokenize_targets(batch):
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch['target'],
            padding="max_length",
            truncation=True,
            max_length=128
        )
    batch['labels'] = labels['input_ids']
    return batch

# Appliquer la tokenisation sur train et validation
tokenized_dataset = dataset.map(tokenize_inputs, batched=True)
tokenized_dataset = tokenized_dataset.map(tokenize_targets, batched=True)

# Supprimer les colonnes inutiles pour l'entraînement
tokenized_dataset = tokenized_dataset.remove_columns(['input', 'target', '__index_level_0__'])

# Vérification d'un échantillon tokenisé
print(tokenized_dataset['train'][0])


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

...

Map:   0%|          | 0/181 [00:00<?, ? examples/s]

Map:   0%|          | 0/1628 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/181 [00:00<?, ? examples/s]

{'input_ids': [1, 6459, 670, 1013, 67, 1880, 39, 67, 17467, 39, 24339, 2428, 12, 1880, 39, 67, 3259, 559, 3262, 14, 4052, 71, 3259, 13, 203, 95, 203, 225, 605, 3118, 67, 6687, 67, 1398, 12, 6687, 21, 1769, 203, 97, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.7 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=baaf48480e03a29d67fb3a87dc70fd717fdc8ff6a3e31e16a058121ae7aaf986
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Charger le tokenizer et le modèle
model_name = "Salesforce/codet5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# 1. Définir les arguments d'entraînement
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=20,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=100,
    save_strategy="epoch",
    report_to="none",  # évite l'avertissement W&B
    fp16=True  # si vous utilisez un GPU compatible
)

# 2. (Optionnel) Fonction de calcul des métriques
import evaluate
metric = evaluate.load("rouge")  # ou autre selon la tâche

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Accessing the relevant scores directly
    # (assuming the result dictionary contains keys like 'rouge1', 'rouge2', 'rougeL')
    return {
        "rouge1": round(result["rouge1"] * 100, 2),
        "rouge2": round(result["rouge2"] * 100, 2),
        "rougeL": round(result["rougeL"] * 100, 2)
    }

# 3. Initialiser le Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 4. Lancer l'entraînement

trainer.train()


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

<ipython-input-15-5d6581a00ceb>:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel
1,1.717900,1.311617,26.360000,20.450000,26.110000
2,1.369400,1.054891,26.310000,20.490000,25.960000
3,1.152900,0.947169,26.610000,21.100000,26.380000
4,1.060000,0.892416,26.460000,20.930000,26.190000
5,1.018700,0.853833,24.470000,19.500000,24.370000
6,0.947200,0.831263,26.260000,20.600000,25.950000
7,0.932200,0.806449,24.470000,19.550000,24.400000
8,0.886900,0.782190,24.570000,19.640000,24.470000
9,0.849200,0.770839,24.110000,19.160000,23.940000
10,0.822300,0.761280,24.450000,19.460000,24.340000


TrainOutput(global_step=4080, training_loss=0.9466471400915407, metrics={'train_runtime': 1127.5062, 'train_samples_per_second': 28.878, 'train_steps_per_second': 3.619, 'total_flos': 4406729055928320.0, 'train_loss': 0.9466471400915407, 'epoch': 20.0})

In [ ]:
from difflib import SequenceMatcher

def compare_test_outputs(expected: str, generated: str, show_diff: bool = True):
    """
    Compare le test attendu et le test généré.
    Affiche la similarité en pourcentage et éventuellement les différences ligne à ligne.
    """
    expected_lines = expected.strip().splitlines()
    generated_lines = generated.strip().splitlines()

    matcher = SequenceMatcher(None, expected.strip(), generated.strip())
    similarity = matcher.ratio() * 100

    print(f"🔍 Similarité : {similarity:.2f}%\n")

    if show_diff:
        print("🔁 Différences ligne à ligne :\n")
        for i, (exp_line, gen_line) in enumerate(zip(expected_lines, generated_lines), start=1):
            if exp_line.strip() != gen_line.strip():
                print(f"❌ Ligne {i} :")
                print(f"   Attendu : {exp_line}")
                print(f"   Généré  : {gen_line}")
        print()

def test_model_on_function(function_code: str, expected_test: str, generated_test: str):
    """
    Affiche les fonctions, tests attendus, générés et effectue la comparaison.
    """
    print("📌 Fonction à tester :\n")
    print(function_code)
    print("\n✅ Test attendu :\n")
    print(expected_test)
    print("\n🤖 Test généré par le modèle :\n")
    print(generated_test)
    print("\n📊 Évaluation :")
    compare_test_outputs(expected_test, generated_test)


In [ ]:
def ton_modele_genere(function_code: str, max_tokens=256) -> str:
    """
    Utilise CodeT5 pour générer un test unitaire à partir d'une fonction C donnée.
    """
    # Préparation du prompt
    prompt = f"Generate a C test function for the following function:\n{function_code}\nTest:\n"

    # Tokenisation
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

    # Move inputs to the same device as the model
    inputs = inputs.to(model.device) # Added this line

    # Génération
    outputs = model.generate(
        **inputs,
        max_length=max_tokens,
        num_beams=4,
        early_stopping=True
    )

    # Décodage de la sortie
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Nettoyage possible
    return generated.replace(prompt, "").strip()

In [ ]:
# Exemple de fonction
function = """
void HAL_GPIO_EXTI_Callback(uint16_t GPIO_Pin)
{
    if(GPIO_Pin == GPIO_PIN_0)
    {
        newButtonState ^= 1;
    }
}
"""

# Test attendu
expected_test = """
void test_HAL_GPIO_EXTI_Callback()
{
    newButtonState = 0;
    HAL_GPIO_EXTI_Callback(GPIO_PIN_0);
    assert(newButtonState == 1);
}
"""

# Génération via le modèle CodeT5
generated_test = ton_modele_genere(function)

# Évaluation
test_model_on_function(function, expected_test, generated_test)


📌 Fonction à tester :


void HAL_GPIO_EXTI_Callback(uint16_t GPIO_Pin)
{
    if(GPIO_Pin == GPIO_PIN_0)
    {
        newButtonState ^= 1;
    }
}


✅ Test attendu :


void test_HAL_GPIO_EXTI_Callback()
{
    newButtonState = 0;
    HAL_GPIO_EXTI_Callback(GPIO_PIN_0);
    assert(newButtonState == 1);
}


🤖 Test généré par le modèle :

```c
void test_HAL_GPIO_EXTI_Callback(void) {
    uint16_t GPIO_Pin = GPIO_PIN_0;
    uint32_t newButtonState = 0;

    HAL_GPIO_EXTI_Callback(GPIO_Pin);
    HAL_GPIO_EXTI_Callback(GPIO_Pin);
    assert(newButtonState == 1);

    HAL_GPIO_EXTI_Callback(GPIO_Pin);
    assert(newButtonState == 1);

📊 Évaluation :
🔍 Similarité : 44.08%

🔁 Différences ligne à ligne :

❌ Ligne 1 :
   Attendu : void test_HAL_GPIO_EXTI_Callback()
   Généré  : ```c
❌ Ligne 2 :
   Attendu : {
   Généré  : void test_HAL_GPIO_EXTI_Callback(void) {
❌ Ligne 3 :
   Attendu :     newButtonState = 0;
   Généré  :     uint16_t GPIO_Pin = GPIO_PIN_0;
❌ Ligne 4 :
   Attendu :     HAL_GPIO_E

In [ ]:
# from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, EarlyStoppingCallback

# # 1. Définir les arguments d’entraînement
# training_args = Seq2SeqTrainingArguments(
#     output_dir="./results",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     weight_decay=0.01,
#     num_train_epochs=10,  # Peut s’arrêter plus tôt grâce à early stopping
#     predict_with_generate=True,
#     logging_dir='./logs',
#     logging_steps=100,
#     save_total_limit=2,
#     load_best_model_at_end=True,
#     metric_for_best_model="rougeL",
#     greater_is_better=True,
#     report_to="none",  # Pour désactiver wandb
#     fp16=True  # si compatible GPU
# )

# # 2. Fonction de calcul des métriques
# import evaluate
# metric = evaluate.load("rouge")

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
#     result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

#     return {
#         "rouge1": round(result["rouge1"] * 100, 2),
#         "rouge2": round(result["rouge2"] * 100, 2),
#         "rougeL": round(result["rougeL"] * 100, 2)
#     }

# # 3. Initialiser le Trainer avec early stopping
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset["train"],
#     eval_dataset=tokenized_dataset["validation"],
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
# )

# # 4. Lancer l’entraînement
# trainer.train()


In [ ]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-small")

# def tokenize(batch):
#     return tokenizer(batch['input'], padding="max_length", truncation=True, max_length=512)

# tokenized_inputs = dataset.map(tokenize, batched=True)

# def tokenize_target(batch):
#     return tokenizer(batch['target'], padding="max_length", truncation=True, max_length=128)

# tokenized_targets = dataset.map(tokenize_target, batched=True)
# tokenized_inputs = tokenized_inputs.add_column("labels", tokenized_targets["input_ids"])


In [ ]:
# from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer

# model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5-small")

# training_args = TrainingArguments(
#     output_dir="./codet5-finetuned",
#     per_device_train_batch_size=8,
#     num_train_epochs=3,
#     save_total_limit=1,
#     save_steps=500,
#     logging_dir='./logs',
#     logging_steps=10,
#     fp16=True,
#     learning_rate=1e-5,

# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_inputs,
#     tokenizer=tokenizer
# )

# trainer.train()
